In [ ]:
# %%capture
# # Install necessary libraries if you don't have them
# !pip install numpy Pillow scikit-image opencv-python scipy pycryptodome

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from scipy.fftpack import dct, idct
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import warnings
import math
import hashlib
import os

# For potential ECC (optional, more advanced)
# from Crypto.Util.number import bytes_to_long, long_to_bytes

warnings.filterwarnings('ignore')

print("🛠️ Robust Block-DCT Watermarking System with QIM")

# --- Helper Functions ---

def text_to_binary(text):
    """Converts a string to its binary representation (list of 0s and 1s)."""
    binary_string = ''.join(format(ord(c), '08b') for c in text)
    return [int(bit) for bit in binary_string]

def binary_to_text(binary_list):
    """Converts a list of 0s and 1s back to a string."""
    binary_string = ''.join(map(str, binary_list))
    # Pad if length is not a multiple of 8
    padding = (8 - len(binary_string) % 8) % 8
    binary_string = binary_string + '0' * padding
    try:
        byte_chunks = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
        ascii_chars = [chr(int(chunk, 2)) for chunk in byte_chunks]
        # Remove potential null characters resulting from padding or errors
        return ''.join(c for c in ascii_chars if c != '\\x00').rstrip('\\x00')
    except ValueError:
        return "[Error decoding binary]" # Handle cases where conversion fails

def calculate_ber(original_bits, extracted_bits):
    """Calculates the Bit Error Rate (BER)."""
    if not original_bits or not extracted_bits:
        return 1.0 # Max error if one list is empty
    compare_len = min(len(original_bits), len(extracted_bits))
    if compare_len == 0:
        return 1.0
    errors = sum(1 for i in range(compare_len) if original_bits[i] != extracted_bits[i])
    # Add errors for length difference if significant cropping happened
    length_diff = abs(len(original_bits) - len(extracted_bits))
    total_errors = errors + length_diff
    # Calculate BER based on the length of the original watermark
    ber = total_errors / len(original_bits)
    return min(ber, 1.0) # BER cannot exceed 1.0

def apply_dct_blocks(image, block_size=8):
    """Applies DCT to each block of the image."""
    h, w = image.shape
    dct_coeffs = np.zeros_like(image, dtype=np.float64)
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            if i + block_size <= h and j + block_size <= w:
                block = image[i:i+block_size, j:j+block_size]
                dct_coeffs[i:i+block_size, j:j+block_size] = dct(dct(block.T, norm='ortho').T, norm='ortho')
    return dct_coeffs

def apply_idct_blocks(dct_coeffs, block_size=8):
    """Applies Inverse DCT to each block."""
    h, w = dct_coeffs.shape
    image_rec = np.zeros_like(dct_coeffs, dtype=np.float64)
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
             if i + block_size <= h and j + block_size <= w:
                block_dct = dct_coeffs[i:i+block_size, j:j+block_size]
                image_rec[i:i+block_size, j:j+block_size] = idct(idct(block_dct.T, norm='ortho').T, norm='ortho')
    return image_rec

# --- QIM Watermarking System ---

class RobustWatermarkQIM:
    def __init__(self, block_size=8, quant_step=20, coeff_pos=(2, 3)):
        """
        Robust Block-DCT watermarking using Quantization Index Modulation (QIM).
        Args:
            block_size (int): Size of the DCT blocks (e.g., 8x8).
            quant_step (float): Quantization step size (delta). Controls robustness vs imperceptibility.
            coeff_pos (tuple): (row, col) index of the DCT coefficient within each block to embed into.
                                Should be mid-frequency, e.g., (2,3), (3,2), (4,1). Avoid (0,0).
        """
        if block_size <= 0 or not isinstance(block_size, int):
             raise ValueError("block_size must be a positive integer.")
        if quant_step <= 0:
             raise ValueError("quant_step must be positive.")
        if not (isinstance(coeff_pos, tuple) and len(coeff_pos) == 2 and
                0 <= coeff_pos[0] < block_size and 0 <= coeff_pos[1] < block_size and coeff_pos != (0,0)):
             raise ValueError(f"coeff_pos must be a valid (row, col) index within the block [0,{block_size-1}], not (0,0).")

        self.block_size = block_size
        self.delta = float(quant_step) # Quantization step
        self.coeff_r, self.coeff_c = coeff_pos # Coefficient position to modify
        print(f"   ⚙️ System Initialized: Block Size={self.block_size}, Delta={self.delta}, Coeff Pos=({self.coeff_r},{self.coeff_c})")

    def _quantize(self, coeff, bit):
        """Applies QIM quantization based on the bit (0 or 1)."""
        # Quantizer centers for bit 0: ..., -delta, 0, delta, 2*delta, ...
        # Quantizer centers for bit 1: ..., -delta/2, delta/2, 3*delta/2, ...
        if bit == 0:
            return round(coeff / self.delta) * self.delta
        else: # bit == 1
            return round((coeff - self.delta / 2) / self.delta) * self.delta + self.delta / 2

    def _dequantize(self, coeff):
        """Decodes the embedded bit using QIM rules."""
        # Find the closest center for bit 0 and bit 1 quantizers
        quantized_0 = round(coeff / self.delta) * self.delta
        quantized_1 = round((coeff - self.delta / 2) / self.delta) * self.delta + self.delta / 2

        dist_0 = abs(coeff - quantized_0)
        dist_1 = abs(coeff - quantized_1)

        # Decode to the bit whose quantizer center is closer
        return 0 if dist_0 <= dist_1 else 1

    def preprocess_image(self, image_path, make_divisible=True):
        """Loads, converts to grayscale, normalizes, and optionally pads/crops."""
        try:
            img = Image.open(image_path).convert('L')
            img_np = np.array(img, dtype=np.float64)

            # Pad or crop to make dimensions divisible by block_size
            if make_divisible:
                h, w = img_np.shape
                new_h = h - (h % self.block_size)
                new_w = w - (w % self.block_size)
                if new_h != h or new_w != w:
                    print(f"   ⚠️ Resizing image from ({h},{w}) to ({new_h},{new_w}) to be divisible by {self.block_size}.")
                    img_np = img_np[:new_h, :new_w] # Simple cropping, padding might be better

            # Normalize to 0-1 range if needed (often DCT is applied on 0-255 or -128 to 127)
            # Let's work in the 0-255 range directly for DCT coefficients
            # img_np = img_np / 255.0
            return np.clip(img_np, 0, 255) # Ensure values are valid pixel range

        except Exception as e:
            print(f"   ❌ Error preprocessing image {image_path}: {e}")
            raise

    def embed_watermark(self, image_path, watermark_text):
        """Embeds the watermark text into the image."""
        print(f"   🔧 Embedding watermark into '{os.path.basename(image_path)}'...")
        try:
            host_image = self.preprocess_image(image_path)
            original_shape = host_image.shape
            watermark_bits = text_to_binary(watermark_text)
            watermark_len = len(watermark_bits)

            h, w = host_image.shape
            num_blocks_h = h // self.block_size
            num_blocks_w = w // self.block_size
            max_capacity = num_blocks_h * num_blocks_w

            if watermark_len > max_capacity:
                print(f"   ⚠️ Warning: Watermark text too long ({watermark_len} bits). Maximum capacity is {max_capacity} bits. Truncating watermark.")
                watermark_bits = watermark_bits[:max_capacity]
                watermark_len = max_capacity
            elif watermark_len == 0:
                 print("   ⚠️ Warning: Watermark text is empty. Nothing to embed.")
                 return host_image # Return original image

            print(f"    Watermark length: {watermark_len} bits")
            print(f"   Image capacity:   {max_capacity} bits")

            # Apply block DCT
            dct_coeffs = apply_dct_blocks(host_image, self.block_size)
            dct_coeffs_modified = dct_coeffs.copy()

            # Embed bits using QIM
            bit_index = 0
            embedded_count = 0
            for i in range(num_blocks_h):
                for j in range(num_blocks_w):
                    if bit_index < watermark_len:
                        block_row, block_col = i * self.block_size, j * self.block_size
                        coeff = dct_coeffs_modified[block_row + self.coeff_r, block_col + self.coeff_c]
                        bit_to_embed = watermark_bits[bit_index]

                        # Apply quantization
                        quantized_coeff = self._quantize(coeff, bit_to_embed)
                        dct_coeffs_modified[block_row + self.coeff_r, block_col + self.coeff_c] = quantized_coeff

                        bit_index += 1
                        embedded_count += 1
                    else:
                        break # Stop if all bits are embedded
                if bit_index >= watermark_len:
                    break

            print(f"   Embedded {embedded_count} bits.")

            # Apply inverse block DCT
            watermarked_image = apply_idct_blocks(dct_coeffs_modified, self.block_size)

            # Clip values to valid image range
            watermarked_image = np.clip(watermarked_image, 0, 255)

            # Calculate PSNR and SSIM
            try:
                psnr_val = psnr(host_image, watermarked_image, data_range=255)
                ssim_val = ssim(host_image, watermarked_image, data_range=255)
                print(f"   📊 PSNR: {psnr_val:.2f} dB")
                print(f"   📊 SSIM: {ssim_val:.4f}")
            except Exception as metric_e:
                print(f"   ⚠️ Could not calculate metrics: {metric_e}")
                psnr_val, ssim_val = None, None


            # Return watermarked image as numpy array and metrics
            return watermarked_image.astype(np.uint8), {'psnr': psnr_val, 'ssim': ssim_val, 'embedded_bits': embedded_count, 'original_shape': original_shape}

        except Exception as e:
            print(f"   ❌ Embedding failed: {e}")
            import traceback
            traceback.print_exc()
            return None, {}

    def extract_watermark(self, image_path, expected_watermark_len):
        """Extracts the watermark bits from a potentially modified image."""
        print(f"   🔎 Extracting watermark from '{os.path.basename(image_path)}'...")
        try:
            # Preprocess without resizing if possible, handle potential size changes
            test_image = self.preprocess_image(image_path, make_divisible=False)
            h, w = test_image.shape
            num_blocks_h = h // self.block_size
            num_blocks_w = w // self.block_size
            available_bits = num_blocks_h * num_blocks_w
            extract_len = min(expected_watermark_len, available_bits)

            if available_bits < expected_watermark_len:
                print(f"   ⚠️ Warning: Image size ({h},{w}) allows extracting only {available_bits} bits, expected {expected_watermark_len}. Potential cropping detected.")
            elif extract_len == 0:
                 print("   ⚠️ Warning: Cannot extract bits from image of this size or expected length is 0.")
                 return [], 0 # Return empty list if no bits can be extracted


            # Apply block DCT to the potentially modified image
            dct_coeffs = apply_dct_blocks(test_image, self.block_size)

            # Extract bits using QIM dequantization
            extracted_bits = []
            bit_index = 0
            for i in range(num_blocks_h):
                for j in range(num_blocks_w):
                    if bit_index < extract_len:
                        block_row, block_col = i * self.block_size, j * self.block_size
                        coeff = dct_coeffs[block_row + self.coeff_r, block_col + self.coeff_c]

                        # Decode bit
                        extracted_bit = self._dequantize(coeff)
                        extracted_bits.append(extracted_bit)
                        bit_index += 1
                    else:
                        break # Stop if expected number of bits are extracted
                if bit_index >= extract_len:
                    break

            print(f"   Extracted {len(extracted_bits)} bits.")
            return extracted_bits, available_bits # Return extracted bits and the max possible for this image size

        except FileNotFoundError:
             print(f"   ❌ Extraction failed: File not found '{image_path}'")
             return None, 0
        except Exception as e:
            print(f"   ❌ Extraction failed: {e}")
            import traceback
            traceback.print_exc()
            return None, 0

    def verify_watermark(self, image_path, original_text, ber_threshold=0.1):
        """Embeds, extracts, and verifies using BER."""
        print("-" * 40)
        print(f"🔍 Verifying '{os.path.basename(image_path)}' against text: '{original_text[:30]}...'")

        original_bits = text_to_binary(original_text)
        if not original_bits:
            print("   ❌ Verification failed: Original text is empty.")
            return False, 1.0, "[Original text empty]", 0

        extracted_bits, available_bits = self.extract_watermark(image_path, len(original_bits))

        if extracted_bits is None:
            print("   ❌ Verification failed due to extraction error.")
            return False, 1.0, "[Extraction Error]", available_bits # Indicate failure

        ber = calculate_ber(original_bits, extracted_bits)
        extracted_text = binary_to_text(extracted_bits)

        print(f"   Original bits len:  {len(original_bits)}")
        print(f"   Extracted bits len: {len(extracted_bits)}")
        print(f"   Available bits:     {available_bits}")
        print(f"   📊 Bit Error Rate (BER): {ber:.4f}")
        # print(f"   Extracted Text (partial): '{extracted_text[:50]}...'") # Optional: print decoded text

        if ber <= ber_threshold:
            print(f"   ✅ Verification PASSED (BER <= {ber_threshold})")
            return True, ber, extracted_text, available_bits
        else:
            print(f"   ❌ Verification FAILED (BER > {ber_threshold})")
            return False, ber, extracted_text, available_bits
        print("-" * 40)


# --- Example Usage & Robustness Tests ---

# --- Create Test Image ---
def create_test_image(filename='lena_gray.png', size=(512, 512)):
    print(f"\n🖼️ Creating/Loading Test Image '{filename}'...")
    if not os.path.exists(filename):
        print("   Generating synthetic image as Lena is not present.")
        # Create a synthetic image if Lena isn't available
        x, y = np.meshgrid(np.linspace(0, 10, size[1]), np.linspace(0, 10, size[0]))
        img_array = (np.sin(x*y/10) + 1) * 127.5 # Example pattern
        img_array = np.clip(img_array, 0, 255).astype(np.uint8)
        img = Image.fromarray(img_array)
        img.save(filename)
    else:
        # Load and ensure grayscale and size
        img = Image.open(filename).convert('L')
        if img.size != size:
             print(f"   Resizing image to {size}")
             img = img.resize(size, Image.Resampling.LANCZOS)
             img.save(filename) # Overwrite with correct size
    print(f"   Using image: {filename} with size {img.size}")
    return filename

# --- Attack Functions ---
def apply_noise(img_array, std_dev=0.01):
    """Adds Gaussian noise (values expected in 0-1 range)."""
    img_float = img_array.astype(np.float64) / 255.0
    noise = np.random.normal(0, std_dev, img_float.shape)
    noisy_img = np.clip(img_float + noise, 0, 1)
    return (noisy_img * 255).astype(np.uint8)

def apply_jpeg_compression(img_array, quality=75):
    """Applies JPEG compression."""
    img_pil = Image.fromarray(img_array)
    buffer = BytesIO()
    img_pil.save(buffer, format='JPEG', quality=quality)
    buffer.seek(0)
    img_compressed = Image.open(buffer).convert('L')
    return np.array(img_compressed)

def apply_cropping(img_array, crop_percent=25):
    """Crops the image from center."""
    h, w = img_array.shape
    crop_h = int(h * crop_percent / 100 / 2)
    crop_w = int(w * crop_percent / 100 / 2)
    cropped = img_array[crop_h:h-crop_h, crop_w:w-crop_w]
    # To simulate finding it later, pad it back (e.g., with zeros or avg color)
    # This simulates finding the cropped part within a larger canvas.
    # For extraction robustness test, we'll test DIRECTLY on the cropped image.
    return cropped

def apply_resizing(img_array, scale_factor=0.5):
    """Resizes down and then up."""
    h, w = img_array.shape
    small_size = (int(w * scale_factor), int(h * scale_factor))
    original_size = (w, h)
    img_small = cv2.resize(img_array, small_size, interpolation=cv2.INTER_AREA)
    img_restored = cv2.resize(img_small, original_size, interpolation=cv2.INTER_LINEAR)
    return img_restored.astype(np.uint8)


# --- Main Test Execution ---
if __name__ == "__main__":

    # --- Configuration ---
    host_image_filename = 'lena_gray_512.png' # Try to use Lena 512x512 grayscale
    image_size = (512, 512)
    watermark_message = "© YourCompanyName 2025. All Rights Reserved. ID: XYZ789"
    watermarked_filename = "watermarked_qim.png"
    attacked_dir = "attacked_images_qim"
    os.makedirs(attacked_dir, exist_ok=True)

    quantization_step = 25 # << --- TRY TUNING THIS (e.g., 10, 20, 30, 40)
    coefficient_position = (3, 4) # << --- TRY TUNING THIS (e.g., (2,3), (4,1), (3,3))
    ber_accept_threshold = 0.15 # Allow slightly higher BER for robustness

    # --- Initialize System ---
    system = RobustWatermarkQIM(quant_step=quantization_step, coeff_pos=coefficient_position)

    # --- Prepare Host Image ---
    host_path = create_test_image(host_image_filename, image_size)
    original_image_np = np.array(Image.open(host_path).convert('L'))

    # --- Embed Watermark ---
    watermarked_image_np, embed_metrics = system.embed_watermark(host_path, watermark_message)

    if watermarked_image_np is not None:
        Image.fromarray(watermarked_image_np).save(watermarked_filename)
        print(f"   ✅ Watermarked image saved as '{watermarked_filename}'")

        # --- Basic Verification (Watermarked Image) ---
        print("\n--- Basic Verification ---")
        passed_basic, ber_basic, _, _ = system.verify_watermark(
            watermarked_filename,
            watermark_message,
            ber_threshold=ber_accept_threshold
        )

        # --- Robustness Tests ---
        print("\n--- Robustness Verification ---")
        robustness_results = {}

        # 1. Noise Attack
        img_noise = apply_noise(watermarked_image_np, std_dev=0.01)
        noise_path = os.path.join(attacked_dir, "attack_noise.png")
        Image.fromarray(img_noise).save(noise_path)
        passed_noise, ber_noise, _, _ = system.verify_watermark(noise_path, watermark_message, ber_threshold=ber_accept_threshold)
        robustness_results["Noise"] = {"Passed": passed_noise, "BER": ber_noise}
        print(f"PSNR (Noise vs WM): {psnr(watermarked_image_np, img_noise, data_range=255):.2f} dB")


        # 2. JPEG Compression Attack
        img_jpeg = apply_jpeg_compression(watermarked_image_np, quality=70) # More aggressive JPEG
        jpeg_path = os.path.join(attacked_dir, "attack_jpeg70.jpg")
        Image.fromarray(img_jpeg).save(jpeg_path)
        passed_jpeg, ber_jpeg, _, _ = system.verify_watermark(jpeg_path, watermark_message, ber_threshold=ber_accept_threshold)
        robustness_results["JPEG (Q=70)"] = {"Passed": passed_jpeg, "BER": ber_jpeg}
        print(f"PSNR (JPEG vs WM): {psnr(watermarked_image_np, img_jpeg, data_range=255):.2f} dB")


        # 3. Cropping Attack (25%)
        img_cropped = apply_cropping(watermarked_image_np, crop_percent=25)
        crop_path = os.path.join(attacked_dir, "attack_crop25.png")
        Image.fromarray(img_cropped).save(crop_path)
        # For cropping, we expect *some* errors, maybe more than the threshold,
        # but the BER shouldn't be 1.0 (total failure). We check if *some* part is detected.
        # We also need to know how many bits *could* have been extracted.
        original_bits = text_to_binary(watermark_message)
        extracted_crop_bits, available_crop_bits = system.extract_watermark(crop_path, len(original_bits))
        if extracted_crop_bits is not None:
            # Calculate BER based on AVAILABLE bits vs corresponding ORIGINAL bits
            crop_h, crop_w = img_cropped.shape
            num_blocks_h_crop = crop_h // system.block_size
            num_blocks_w_crop = crop_w // system.block_size
            expected_extract_len = min(len(original_bits), num_blocks_h_crop * num_blocks_w_crop)

            # Compare only the bits we could extract
            ber_crop = calculate_ber(original_bits[:expected_extract_len], extracted_crop_bits[:expected_extract_len])
            passed_crop = ber_crop <= ber_accept_threshold
            print(f"   📊 Bit Error Rate (BER) [Cropped Portion]: {ber_crop:.4f}")
            if passed_crop:
                 print(f"   ✅ Cropping Verification PASSED (BER <= {ber_accept_threshold})")
            else:
                 print(f"   ⚠️ Cropping Verification FAILED (BER > {ber_accept_threshold}) - But some bits extracted.")

            robustness_results["Cropping (25%)"] = {"Passed": passed_crop, "BER (partial)": ber_crop, "Available": available_crop_bits, "Expected": len(original_bits)}
        else:
             print("   ❌ Cropping Verification FAILED (Extraction Error)")
             robustness_results["Cropping (25%)"] = {"Passed": False, "BER": 1.0}


        # 4. Resizing Attack
        img_resized = apply_resizing(watermarked_image_np, scale_factor=0.5)
        resize_path = os.path.join(attacked_dir, "attack_resize50.png")
        Image.fromarray(img_resized).save(resize_path)
        passed_resize, ber_resize, _, _ = system.verify_watermark(resize_path, watermark_message, ber_threshold=ber_accept_threshold)
        robustness_results["Resizing (50%)"] = {"Passed": passed_resize, "BER": ber_resize}
        print(f"PSNR (Resize vs WM): {psnr(watermarked_image_np, img_resized, data_range=255):.2f} dB")


        # 5. Tampering Simulation (Replace a block)
        img_tampered = watermarked_image_np.copy()
        tamper_size = 64
        h, w = img_tampered.shape
        # Replace a central block with noise
        start_h, start_w = h//2 - tamper_size//2, w//2 - tamper_size//2
        img_tampered[start_h:start_h+tamper_size, start_w:start_w+tamper_size] = np.random.randint(0, 256, (tamper_size, tamper_size), dtype=np.uint8)
        tamper_path = os.path.join(attacked_dir, "attack_tamper.png")
        Image.fromarray(img_tampered).save(tamper_path)
        passed_tamper, ber_tamper, _, _ = system.verify_watermark(tamper_path, watermark_message, ber_threshold=ber_accept_threshold)
        robustness_results["Tampering (Block Replace)"] = {"Passed": passed_tamper, "BER": ber_tamper}
        print(f"PSNR (Tamper vs WM): {psnr(watermarked_image_np, img_tampered, data_range=255):.2f} dB")


        # --- Final Summary ---
        print("\n--- ROBUSTNESS SUMMARY ---")
        print(f"{'Attack':<25} | {'Passed':<7} | {'BER':<15}")
        print("-" * 50)
        for name, res in robustness_results.items():
            ber_str = f"{res.get('BER', 'N/A'):.4f}" if isinstance(res.get('BER'), float) else f"{res.get('BER (partial)', 'N/A'):.4f} (partial)"
            print(f"{name:<25} | {str(res['Passed']):<7} | {ber_str:<15}")

        # --- Verify Original Image (Should Fail) ---
        print("\n--- Control Verification (Original Image) ---")
        passed_orig, ber_orig, _, _ = system.verify_watermark(
            host_path,
            watermark_message,
            ber_threshold=ber_accept_threshold
        )

    else:
        print("Embedding failed, skipping verification and robustness tests.")